In [ ]:
%matplotlib notebook

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import json
from datetime import datetime
from collections import namedtuple

In [ ]:
Message = namedtuple('Message', ['timestamp', 'body', 'author'])

def msg_date(m):
    return datetime.fromtimestamp(m['timestamp']/1000)

def auth(a):
    return a
#     return {
#         'their_id': 'their_name',
#         'your_id': 'your_name',
#         'their_name': 'their_name',
#         'your_name': 'your_name'
#     }[a]

def scrplay(msg):
    return "{}: {}".format(auth(msg['author']), msg['body'])

In [ ]:
data = json.loads(open('../../data/complete.min.json', 'r').read())

data = [
    Message(
        timestamp=d['timestamp'], 
        body=d['body'], 
        author=auth(d['author']))
    for d in data
]

## Clustering Conversations

To cluster by conversation, we have to know what the time delay between messages is, and what constitutes a new 'conversation'. To do this, simply plot time-since-last-message.

In [ ]:
times = []
n_back = 2
for m in range(n_back, len(data[n_back:])):
    times.append((data[m].timestamp - data[m - 1].timestamp) / data[m - 2].timestamp)
    
times.remove(np.max(times));times.remove(np.max(times));

plt.figure("Conversation Clusters")
plt.xlim(-.0001, .002)
plt.scatter(times, [1] * len(times))
plt.show()

In [ ]:
TIME_DELIM = 60 * 60 * 1000 # 30 minute cutoff between conversations

In [ ]:
conversations = [[data[0]]]
last_msg = data[0]
for msg in data[1:]:
    if msg.timestamp - last_msg.timestamp < TIME_DELIM:
        conversations[-1].append(msg)
    else:
        conversations.append([msg])
    last_msg = msg

print("{} segmented conversations, average length {}.".format(
        len(conversations),
        np.mean([len(c) for c in conversations])
    ))

In [ ]:
print("{} conversations per day.".format(len(conversations)/339))

SHORT_CUTOFF = 10
short_convos = [c for c in conversations if len(c) < SHORT_CUTOFF]
print("{} conversations are less than {} messages long.".format(len(short_convos), SHORT_CUTOFF))

complex_conversations = [c for c in conversations if c not in short_convos]
print(len(complex_conversations))


In [ ]:
xs = [c[0].timestamp for c in complex_conversations]
ys = [len(c) for c in complex_conversations]

# plt.scatter(xs, ys)
i = 0
for c in complex_conversations[400:]:
    i += 1 #randomize + or -, so get prettier image
    plt.plot([c[0].timestamp, c[0].timestamp + ((c[-1].timestamp - c[0].timestamp))], [1,1], linewidth=2)